# ConvNet Tutorial

**This lesson is adapted from [sentdex](https://www.youtube.com/watch?v=WvoLTXIjBYU&t=1s) on YouTube. I have collated this for my own learning experience, as well as for the benefit of others who would like to learn as well. :)**

Before we start, make sure you have a basic understanding of how convolutional neural networks work. You can read about it [here](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53), or watch sentdex's video explanation on YouTube.

First, let us import all the necessary modules that we need. Some new modules you will see here are Dropout, Conv2D and MaxPooling 2D.

Dropout helps against the overfitting of our model by randomly setting the outgoing edges of hidden units (neurons that make up hidden layers) to 0 at each update of the training phase.

Conv2D is a 2D Convolution Layer that creates a convolution kernel that is wind with layers input which helps produce a tensor of outputs. 

MaxPooling2D is a maxpooling operation for 2D spatial data. 

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import numpy as np
import pickle

Next, we will load the data that we have saved from the previous lesson.

In [16]:
x = pickle.load(open("x.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

#normalise the data
#since the maximum pixel value is 255, we will divide our x by 255 to achieve values between 0 and 1
x = x/255.0
y = np.array(y)

Now, we will start training our model. 

We can input almost anything for the first value of the convolutional 2D layer, the second value would be the window that we would like to use (3 by 3), and for the input shape, we will pass in the shape of our x, while skipping -1.

Next, we can either go through with maxpooling or activation first, it doesn't matter. For activation, we will be using the widey-used [relu function](https://medium.com/@danqing/a-practical-guide-to-relu-b83ca804f1f7#:~:text=ReLU%20stands%20for%20rectified%20linear,max(0%2C%20x).&text=ReLU%20is%20the%20most%20commonly,usually%20a%20good%20first%20choice.).

For this model, the dense layer is not needed, but we will add it in anyway for good measure.

In [19]:
#using the sequential model
model = Sequential()

#first layer
model.add(Conv2D(64, (3, 3), input_shape = x.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

#second layer
model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

#third layer
#first we need to flatten the dataset because conv2d pass through a 2D dataset, whereas dense accepts a 1D dataset
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

#output layer with sigmoid function as activation function
model.add(Dense(1))
model.add(Activation('sigmoid'))

#specify the optimizer, loss function and metrics(optional)
#loss function here is binary crossentropy because we only have 2 outputs (cat or dog)
model.compile(loss = "binary_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])

Since our data size this time a very large (over 20000 images if I'm not wrong), passing our data 1 by 1 would prove to be too slow, and passing them all at once might cause the code to run very slowly. As a result, we will be passing them through as batches. Usually, a good batch size would be around 20, but it also depends on how big or how small your dataset is.

To prevent the dataset from overfitting, we will be splittiing some of the training data intro validation data. If you would like to learn more about the importance of validation data, you can read so [here](https://machinelearningmastery.com/difference-test-validation-datasets/).

In [20]:
model.fit(x, y, batch_size = 32, epochs = 10, validation_split = 0.1)

Epoch 1/10
702/702 [==============================] - 66s 93ms/step - loss: 0.6302 - accuracy: 0.6372 - val_loss: 0.5660 - val_accuracy: 0.7094
Epoch 2/10
702/702 [==============================] - 63s 90ms/step - loss: 0.5228 - accuracy: 0.7414 - val_loss: 0.4859 - val_accuracy: 0.7607
Epoch 3/10
702/702 [==============================] - 71s 101ms/step - loss: 0.4801 - accuracy: 0.7697 - val_loss: 0.4629 - val_accuracy: 0.7792
Epoch 4/10
702/702 [==============================] - 67s 96ms/step - loss: 0.4469 - accuracy: 0.7907 - val_loss: 0.4530 - val_accuracy: 0.7844
Epoch 5/10
702/702 [==============================] - 63s 89ms/step - loss: 0.4129 - accuracy: 0.8091 - val_loss: 0.4484 - val_accuracy: 0.7884
Epoch 6/10
702/702 [==============================] - 65s 92ms/step - loss: 0.3821 - accuracy: 0.8251 - val_loss: 0.4481 - val_accuracy: 0.7896
Epoch 7/10
702/702 [==============================] - 66s 95ms/step - loss: 0.3494 - accuracy: 0.8461 - val_loss: 0.4329 - val_accuracy

In [21]:
#save a model
model.save('dogs_cats_predict.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: dogs_cats_predict.model\assets


In [ ]:
#load a model
new_model = tf.keras.models.load_model('dogs_cats_predict.model')